
# Tenseal Playground
So since this is not going to be a part of the git repository I'm not going to be formal in here. So lets go on and do the exploration.

## Should we use CKKS?
No since then we should handle the noise and then do the ZKP, hence it would lead to nasty stuff. So we should use the BFV scheme. Also the BFV scheme is faster and we can find the upper limit for our operations.

Since we are going to calculate the variance of the data we need to have:

$$ \sigma^2 = \sum_{i=1}^{n} x_i^2 - \frac{1}{n}(\sum_{i=1}^{n} x_i)^2 $$

So we need two things:

$$ \sum_{i=1}^{n} x_i^2 $$

$$ \sum_{i=1}^{n} x_i $$


Then the rest is easy. We can calculate the variance after the decryption.

We then need to add bias and random number to it. So we can do the ZKP.

Considering that we want to do the process for 100 meters of the street and the length of each car being 3.5 to 5 meters and also 0.5 meters for the space between the cars. We can say that we have at most 25 cars per each lane and considering at most four lanes the result would be 100 cars. So per each phase the maximum number we need to deal with is in terms of `cl` the car length, and `s` the space between the cars and `sl` street length is:

number of cars = street length / car length + space between cars)

Maximum number would be for the sum of the squares of the speeds with the bias and maximum random number possible. So the number would be:

max number  = number of cars * max speed^2 + bias + random number

Also the bias and the random number must be the same hence:

max number = number of cars * max speed^2 + 2 * bias

So we can calculate the upper limit for the operations.

max number = 100 * 100^2 +  2 * 10_000 = 1_020_000 

So this would be the plain text upper limit for the operations.

In [ ]:
import tenseal as ts

context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
context

Lets encrypt a list of numbers and then decrypt them.

In [ ]:
plain = [1, 2, 3, 4, 5]
encrypted_vector = ts.bfv_vector(context, plain)
print("Encrypted vector size is: ", encrypted_vector.size())
print("Encrypted vector: ", encrypted_vector.decrypt())

Let's do serializations.

In [80]:
import base64

def b64_enc(data):
    return base64.b64encode(data).decode('utf-8')

def b64_dec(data):
    return base64.b64decode(data)

ev = encrypted_vector.serialize()
evb64 = b64_enc(ev)
print(type(evb64))
evb64


<class 'str'>


'...'

In [ ]:
print(type(encrypted_vector.serialize()))
print(type(ev))
ev == b64_dec(evb64)

In [ ]:
ev = encrypted_vector.serialize().hex()
len(ev)

In [ ]:
pvc = context.serialize(save_secret_key=True).hex()
print(f'Is context public: {context.is_public()}')
len(pvc)

In [ ]:
context.make_context_public()
puc = context.serialize().hex()
len(puc)

Now lets deserialize stuff

In [ ]:
context2 = ts.context_from(bytes.fromhex(puc))
print(f'Is context public: {context2.is_public()}')
context2

In [ ]:
encrypted_vector2 = ts.bfv_vector_from(context2, bytes.fromhex(ev))
print("Encrypted vector size is: ", encrypted_vector2.size())

In [ ]:
context3 = ts.context_from(bytes.fromhex(pvc))
print(context3.is_public())

In [ ]:
encrypted_vector3 = ts.bfv_vector_from(context3, bytes.fromhex(ev))
print("Encrypted vector size is: ", encrypted_vector3.size())

In [ ]:
encrypted_vector3.decrypt()

But we don't need to do the bias. We can just add and remove the error. We also don't need to check the boundary of the FHE limit. The original number will eventually be reached


In [81]:
ctx_str = '...'

ctx = ts.context_from(b64_dec(ctx_str))

In [82]:
ts.bfv_vector(ctx, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])